# Урок 8. Рекуррентные нейронные сети RNN LSTM GRU

# Домашнее задание

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import tensorflow_addons as tfa

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping 

In [4]:
df = pd.read_excel('summer_reviews.xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [5]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

In [6]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [7]:
df_train.head()

,Rating,Content,Date
3517,5,наконецтый исправить чушь снеоргинальный проши...,2017-08-09
12399,5,удобно использование,2017-07-27
3833,5,отлично,2017-08-08
13250,5,класс,2017-07-25
17941,5,удобно,2017-07-08


In [8]:
text_corpus_train = df_train['Content'].values
text_corpus_test = df_test['Content'].values

tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n', #символы для удаления из текстов
                     lower = False, #перевод к нижнему регистру
                      split = ' ') #разделитель для слов

tokenizer.fit_on_texts(text_corpus_train)

len(tokenizer.index_word)

10292

In [9]:
sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

sequences_train[:10]

[[610, 42, 775, 3539, 22, 1, 2, 15],
 [2, 181],
 [5],
 [60],
 [2],
 [127, 1],
 [37, 970, 66, 104, 30, 484, 887],
 [102, 51, 1, 690, 139, 732, 106, 1968, 53, 1969, 1625, 2],
 [463, 206, 147, 128, 99, 20, 111, 1059, 51, 9],
 [26, 1371, 888, 5, 3540, 733]]

In [10]:
word_count = len(tokenizer.index_word) + 1 
training_length = max([len(i.split()) for i in text_corpus_train])
word_count, training_length 

(10293, 113)

In [11]:
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [12]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)

#### CONV

In [13]:
model = Sequential()
model.add(Embedding(input_dim=word_count, output_dim=128, input_length=training_length))                   
model.add(Conv1D(128, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 113, 128)          1317504   
                                                                 
 conv1d (Conv1D)             (None, 111, 128)          49280     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 activation (Activation)     (None, 5)                 0         
                                                        

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [15]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True) 

epochs = 20
batch_size = 512

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/20
25/25 [==============================] - 13s 27ms/step - loss: 1.0343 - accuracy: 0.6795 - f1_score: 0.1860 - val_loss: 0.8221 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 0s 13ms/step - loss: 0.7257 - accuracy: 0.7498 - f1_score: 0.2751 - val_loss: 0.7291 - val_accuracy: 0.7430 - val_f1_score: 0.2771
Epoch 3/20
25/25 [==============================] - 0s 13ms/step - loss: 0.6318 - accuracy: 0.7748 - f1_score: 0.2999 - val_loss: 0.7032 - val_accuracy: 0.7466 - val_f1_score: 0.2875
Epoch 4/20
25/25 [==============================] - 0s 12ms/step - loss: 0.5680 - accuracy: 0.7943 - f1_score: 0.3417 - val_loss: 0.7006 - val_accuracy: 0.7509 - val_f1_score: 0.3153
Epoch 5/20
25/25 [==============================] - 0s 13ms/step - loss: 0.5089 - accuracy: 0.8186 - f1_score: 0.3982 - val_loss: 0.7015 - val_accuracy: 0.7531 - val_f1_score: 0.3365
Epoch 6/20
25/25 [==============================] - 0s 13ms/step - loss: 0.4526 - ac

In [17]:
score_conv = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 0s 6ms/step - loss: 1.3426 - accuracy: 0.7353 - f1_score: 0.3829


#### SimpleRNN

In [18]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 113, 30)           308790    
                                                                 
 masking (Masking)           (None, 113, 30)           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                6080      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
Total params: 319,355
Trainable params: 319,355
Non-tr

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [20]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/20
25/25 [==============================] - 7s 203ms/step - loss: 1.1234 - accuracy: 0.6830 - f1_score: 0.1850 - val_loss: 0.9707 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 4s 175ms/step - loss: 0.9140 - accuracy: 0.7084 - f1_score: 0.1962 - val_loss: 0.8525 - val_accuracy: 0.7155 - val_f1_score: 0.2633
Epoch 3/20
25/25 [==============================] - 4s 174ms/step - loss: 0.7801 - accuracy: 0.7425 - f1_score: 0.2754 - val_loss: 0.7652 - val_accuracy: 0.7271 - val_f1_score: 0.2555
Epoch 4/20
25/25 [==============================] - 5s 180ms/step - loss: 0.6936 - accuracy: 0.7680 - f1_score: 0.3012 - val_loss: 0.7354 - val_accuracy: 0.7329 - val_f1_score: 0.2750
Epoch 5/20
25/25 [==============================] - 4s 179ms/step - loss: 0.6193 - accuracy: 0.7855 - f1_score: 0.3323 - val_loss: 0.7197 - val_accuracy: 0.7502 - val_f1_score: 0.3069
Epoch 6/20
25/25 [==============================] - 5s 182ms/step - loss: 0.5563

In [21]:
score_srnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 1s 41ms/step - loss: 1.1393 - accuracy: 0.7364 - f1_score: 0.3651


#### LSTM

In [22]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 113, 30)           308790    
                                                                 
 masking_1 (Masking)         (None, 113, 30)           0         
                                                                 
 lstm (LSTM)                 (None, 64)                24320     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                                 
Total params: 337,595
Trainable params: 337,595
Non-tr

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [24]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/20
25/25 [==============================] - 15s 506ms/step - loss: 1.2392 - accuracy: 0.6778 - f1_score: 0.1755 - val_loss: 0.9476 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 12s 481ms/step - loss: 0.8857 - accuracy: 0.7030 - f1_score: 0.1654 - val_loss: 0.8164 - val_accuracy: 0.7040 - val_f1_score: 0.1803
Epoch 3/20
25/25 [==============================] - 12s 481ms/step - loss: 0.7743 - accuracy: 0.7369 - f1_score: 0.2586 - val_loss: 0.7452 - val_accuracy: 0.7336 - val_f1_score: 0.2667
Epoch 4/20
25/25 [==============================] - 12s 477ms/step - loss: 0.6987 - accuracy: 0.7633 - f1_score: 0.2880 - val_loss: 0.7164 - val_accuracy: 0.7350 - val_f1_score: 0.2693
Epoch 5/20
25/25 [==============================] - 12s 482ms/step - loss: 0.6395 - accuracy: 0.7759 - f1_score: 0.3022 - val_loss: 0.7090 - val_accuracy: 0.7480 - val_f1_score: 0.2874
Epoch 6/20
25/25 [==============================] - 12s 477ms/step - loss: 

In [25]:
score_lstm = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 2s 112ms/step - loss: 1.1751 - accuracy: 0.7413 - f1_score: 0.3910


#### GRU

In [26]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64,recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 113, 30)           308790    
                                                                 
 masking_2 (Masking)         (None, 113, 30)           0         
                                                                 
 gru (GRU)                   (None, 64)                18432     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 5)                 325       
                                                                 
Total params: 331,707
Trainable params: 331,707
Non-tr

In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [28]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/20
25/25 [==============================] - 14s 454ms/step - loss: 1.3272 - accuracy: 0.6732 - f1_score: 0.1794 - val_loss: 0.9789 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 11s 443ms/step - loss: 0.8690 - accuracy: 0.7030 - f1_score: 0.1651 - val_loss: 0.8134 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 3/20
25/25 [==============================] - 11s 428ms/step - loss: 0.7560 - accuracy: 0.7205 - f1_score: 0.2261 - val_loss: 0.7489 - val_accuracy: 0.7155 - val_f1_score: 0.2348
Epoch 4/20
25/25 [==============================] - 11s 433ms/step - loss: 0.6811 - accuracy: 0.7661 - f1_score: 0.2892 - val_loss: 0.7268 - val_accuracy: 0.7336 - val_f1_score: 0.2683
Epoch 5/20
25/25 [==============================] - 11s 431ms/step - loss: 0.6253 - accuracy: 0.7802 - f1_score: 0.3027 - val_loss: 0.7201 - val_accuracy: 0.7422 - val_f1_score: 0.2780
Epoch 6/20
25/25 [==============================] - 11s 450ms/step - loss: 

In [29]:
score_gru= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 1s 39ms/step - loss: 1.2427 - accuracy: 0.7314 - f1_score: 0.3782


In [30]:
result_RNN = pd.DataFrame({'model': ['SimpleRNN', 'LSTM', 'GRU'], 'f1_score': [score_srnn[1], score_lstm[1], score_gru[1]]})
result_RNN.sort_values(by='f1_score', ascending=False)

,model,f1_score
1,LSTM,0.741273
0,SimpleRNN,0.736433
2,GRU,0.731446


#### LSTM работает лучше всех, GRU - хуже всех

#### CNN -> RNN

In [31]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64,recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 113, 30)           308790    
                                                                 
 masking_3 (Masking)         (None, 113, 30)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 111, 128)          11648     
                                                                 
 activation_1 (Activation)   (None, 111, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)               

In [33]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [35]:
%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs
Epoch 1/20
25/25 [==============================] - 11s 430ms/step - loss: 0.7312 - accuracy: 0.7586 - f1_score: 0.2848 - val_loss: 0.7293 - val_accuracy: 0.7394 - val_f1_score: 0.2731
Epoch 2/20
25/25 [==============================] - 11s 430ms/step - loss: 0.6446 - accuracy: 0.7768 - f1_score: 0.3023 - val_loss: 0.7209 - val_accuracy: 0.7430 - val_f1_score: 0.2784
Epoch 3/20
25/25 [==============================] - 11s 424ms/step - loss: 0.5827 - accuracy: 0.7913 - f1_score: 0.3308 - val_loss: 0.7290 - val_accuracy: 0.7545 - val_f1_score: 0.3287
Epoch 4/20
25/25 [==============================] - 12s 475ms/step - loss: 0.5368 - accuracy: 0.8104 - f1_score: 0.3820 - val_loss: 0.7382 - val_accuracy: 0.7495 - val_f1_score: 0.3282
Epoch 5/20
25/25 [==============================] - 11s 431ms/step - loss: 0.4962 - accuracy: 0.8271 - f1_score: 0.4259 - val_loss: 0.7654 - val_accuracy: 0.7437 - val_f1_score: 0.3570
Epoch 6/20


In [36]:
score_cnnrnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 1s 51ms/step - loss: 1.3746 - accuracy: 0.7212 - f1_score: 0.3836


#### RNN -> CNN

In [38]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(64, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 113, 30)           308790    
                                                                 
 masking_5 (Masking)         (None, 113, 30)           0         
                                                                 
 lstm_3 (LSTM)               (None, 113, 64)           24320     
                                                                 
 conv1d_3 (Conv1D)           (None, 111, 64)           12352     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                      

In [39]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [40]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/20
25/25 [==============================] - 56s 545ms/step - loss: 1.2495 - accuracy: 0.6798 - f1_score: 0.1723 - val_loss: 1.0248 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 14s 554ms/step - loss: 0.9954 - accuracy: 0.7030 - f1_score: 0.1651 - val_loss: 0.9195 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 3/20
25/25 [==============================] - 12s 496ms/step - loss: 0.8273 - accuracy: 0.7260 - f1_score: 0.2512 - val_loss: 0.7706 - val_accuracy: 0.7329 - val_f1_score: 0.2684
Epoch 4/20
25/25 [==============================] - 12s 496ms/step - loss: 0.7306 - accuracy: 0.7579 - f1_score: 0.2867 - val_loss: 0.7335 - val_accuracy: 0.7401 - val_f1_score: 0.2752
Epoch 5/20
25/25 [==============================] - 12s 494ms/step - loss: 0.6820 - accuracy: 0.7706 - f1_score: 0.2988 - val_loss: 0.7230 - val_accuracy: 0.7379 - val_f1_score: 0.2734
Epoch 6/20
25/25 [==============================] - 13s 507ms/step - loss: 

In [41]:
score_rnncnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 2s 131ms/step - loss: 1.0596 - accuracy: 0.7420 - f1_score: 0.3803


In [42]:
result_CNN_RNN = pd.DataFrame({'model': ['CNN_RNN', 'RNN_CNN'], 'f1_score': [score_cnnrnn[1], score_rnncnn[1]]})
result_CNN_RNN.sort_values(by='f1_score', ascending=False)

,model,f1_score
1,RNN_CNN,0.742006
0,CNN_RNN,0.721179


#### У RNN_CNN результат лучше

### Сравнение всех моделей

In [43]:
result_CNN = pd.DataFrame({'model': 'CNN', 'f1_score': [score_conv[1]]})
result_models = pd.concat([result_CNN, result_RNN, result_CNN_RNN], ignore_index=True)
result_models.sort_values(by='f1_score', ascending=False)

,model,f1_score
5,RNN_CNN,0.742006
2,LSTM,0.741273
1,SimpleRNN,0.736433
0,CNN,0.735260
3,GRU,0.731446
4,CNN_RNN,0.721179
